# Домашнее задание
Почувствуй мощь трансформеров в бою

Описание/Пошаговая инструкция выполнения домашнего задания:

1. В качестве данных выберете возьмите датасет RuCola для русского языка https://github.com/RussianNLP/RuCoLA (в качестве train возьмите in_domain_train.csv, а в качестве теста in_domain_dev.csv)

2. Разбейте in_domain_train на train и val

3. Зафайнтьюньте и протестируйте RuBert или RuRoBerta на данной задаче (можно взять любую предобученную модель руберт с сайта huggingface.            
Например:
- ruBert-base/large https://huggingface.co/sberbank-ai/ruBert-base / https://huggingface.co/sberbank-ai/ruBert-large
- rubert-base-cased https://huggingface.co/DeepPavlov/rubert-base-cased, ruRoberta-large https://huggingface.co/sberbank-ai/ruRoberta-large, xlm-roberta-base https://huggingface.co/xlm-roberta-base)
                                                     
4. Возьмите RuGPT3 base или large и решите данное задание с помощью методов few-/zero-shot

а) переберите несколько вариантов затравок

б) протестируйте различное число few-shot примеров (0, 1, 2, 4)

5. Обучите и протестируйте модель RuT5 на данной задаче (пример finetun’а можете найти здесь https://github.com/RussianNLP/RuCoLA/blob/main/baselines/finetune_t5.py)

6.Сравните полученные результаты

In [2]:
!git clone https://github.com/RussianNLP/RuCoLA.git

Cloning into 'RuCoLA'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 73 (delta 30), reused 52 (delta 22), pack-reused 0
Receiving objects: 100% (73/73), 944.21 KiB | 8.14 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [3]:
!pip install transformers

In [4]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler

from tqdm import tqdm

from sklearn.metrics import f1_score

# 1.Подготовка данных
в качестве train возьмем in_domain_train.csv, а в качестве теста in_domain_dev.csv

In [5]:
train = pd.read_csv('in_domain_train.csv', usecols=[1,2])
train.head(5)

,sentence,acceptable
0,"Вдруг решетка беззвучно поехала в сторону, и н...",1
1,Этим летом не никуда ездили.,0
2,Только Иван выразил какую бы то ни было готовн...,1
3,"Теперь ты видишь собственными глазами, как тут...",1
4,На поверку вся теория оказалась полной чепухой.,1


In [6]:
test = pd.read_csv('in_domain_dev.csv', usecols=[1,2])
test.head(3)

,sentence,acceptable
0,Иван вчера не позвонил.,1
1,"У многих туристов, кто посещают Кемер весной, ...",0
2,Лесные запахи набегали волнами; в них смешалос...,1


In [7]:
# Разделения набора данных train (in_domain_train) на обучающую и валидационную выборки
idx = train.sample(frac=0.8, random_state=42).index
               # *.index - После выполнения метода sample, получаем объект DataFrame, и .index возвращает индексы выбранных строк.
               # Эти индексы сохраняются в переменной idx.
val = train[~train.index.isin(idx)] # Используя инвертированный булев массив, выбираем строки из train, индексы которых не входят в idx.
               # Cоздаем валидационную выборку val, содержащую оставшиеся 20% данных.
train = train[train.index.isin(idx)] # - в train оставляем только те строки, индексы которых находятся в idx (остаются только 80% данных,
                                     # которые были выбраны для обучения)

del idx

print('Train size:', len(train))
print('Val size:', len(val))
print('Test size:', len(test))
print('\n')
print('Train labels counts\n', train['acceptable'].value_counts().to_dict(), '\n')
print('Eval labels counts\n', val['acceptable'].value_counts().to_dict(), '\n')
print('Test labels counts\n', test['acceptable'].value_counts().to_dict(), '\n')

Train size: 6295
Val size: 1574
Test size: 983


Train labels counts
 {1: 4690, 0: 1605} 

Eval labels counts
 {1: 1174, 0: 400} 

Test labels counts
 {1: 733, 0: 250} 



# 2. Файнтьюн и тестирование модели ruBERT на данной задаче

In [8]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Инициализация переменной max_len (считаем максимальое количество токенов в предложении)
max_len = 0

# Цикл по предложениям
for sent in train['sentence']:

    # Токенизация предложения и добавление токенов:[CLS]-(класс) в начале и [SEP] (разделитель) в конце
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Обновление максимальной длины предложения
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  45


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device);

cuda


In [11]:
# Определение класса EvalDataset- предназначен для оценки/тестирования модели
class EvalDataset(Dataset):

    def __init__(self, X):
        self.text = X.reset_index(drop=True)

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=45)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1).to(device) for k, v in output.items()}

# Определение класса TrainDataset
class TrainDataset(Dataset):

    def __init__(self, X, label):
        self.text = X.reset_index(drop=True)
        self.label = label.reset_index(drop=True)

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=45)

    def __len__(self):
        return self.label.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        output.update({'labels': torch.tensor(self.label[index])})
        return {k: v.reshape(-1).to(device) for k, v in output.items()}

# Создание экземпляров датасетов и загрузчиков данных для пакетной обработки с заданными параметрами
train_ds = TrainDataset(train['sentence'], train['acceptable'])
train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True)

eval_ds = TrainDataset(val['sentence'], val['acceptable'])
eval_dataloader = DataLoader(eval_ds, batch_size=32)

test_ds = EvalDataset(test['sentence'])
test_dataloader = DataLoader(test_ds, batch_size=32)

In [12]:
# Определение оптимизатора
optimizer = Adam(model.parameters(), lr=5e-6) # model.parameters-возвращает веса и смещения модели, которые будут обновляться во время обучения.
                                              # Оптимизатор будет использовать эти параметры для вычисления обновлений.
# Определение количества эпох и шагов обучения
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader) # Вычисляет общее количество шагов обучения.
             # len(train_dataloader) возвращает количество батчей в обучающем наборе данных.
             # Умножив это число на количество эпох, мы получаем общее количество обновлений параметров модели, которое будет выполнено в процессе обучения
# Определение планировщика скорости обучения. Планировщик позволяет изменять скорость обучения в процессе обучения, что может помочь улучшить сходимость модели.
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

### Обучение Train Loop

In [13]:
def train_model(train_dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} \n **************')
         # Forward pass (скормить данные в нейросеть и пробросить вперед)
        for n_batch, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            if n_batch % 50 == 0:
                loss_value, current = loss.item(), n_batch * batch['input_ids'].shape[0]
                print(f"Loss train: {loss_value:>7f}  [{current:>5d}/{len(train_ds):>5d}]")
                print('Evaluating...')
                preds, true = test_model(eval_dataloader, eval=True)
                print(f'F1-score = {f1_score(preds, true):>3f}\n')
            # Backward pass (backpropagation - посчитаем градиенты по всем параметрам с помощью обратного распространения ошибки)
            loss.backward()
            # Обновляем параметры с помощью optimizer.step()
            optimizer.step()
            lr_scheduler.step()
            # зануляем градиенты предыдущего шага
            optimizer.zero_grad()

def test_model(test_dataloader, eval=False):
    model.eval()
    y_pred = np.array([])
    y_true = np.array([])
    for n_batch, batch in enumerate(test_dataloader):
        if eval:
            y_true = np.hstack([y_true, batch['labels'].cpu().numpy().reshape(-1)])
        outputs = model(**batch)
        y_pred = np.hstack([y_pred, outputs['logits'].argmax(axis=1).detach().cpu().numpy()])
    return y_pred, y_true

In [14]:
train_model(train_dataloader, num_epochs)

Epoch 1 
 **************
Loss train: 0.706457  [    0/ 6295]
Evaluating...
F1-score = 0.549416

Loss train: 0.614599  [ 1600/ 6295]
Evaluating...
F1-score = 0.854023

Loss train: 0.464013  [ 3200/ 6295]
Evaluating...
F1-score = 0.853916

Loss train: 0.707874  [ 4800/ 6295]
Evaluating...
F1-score = 0.854645

Epoch 2 
 **************
Loss train: 0.615815  [    0/ 6295]
Evaluating...
F1-score = 0.855162

Loss train: 0.533991  [ 1600/ 6295]
Evaluating...
F1-score = 0.855162

Loss train: 0.443596  [ 3200/ 6295]
Evaluating...
F1-score = 0.854525

Loss train: 0.484330  [ 4800/ 6295]
Evaluating...
F1-score = 0.856722



In [15]:
y_pred, _ = test_model(test_dataloader, eval=False)
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.862374



In [16]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_pred, test["acceptable"])
print('Accuracy: %.3f' % (accuracy))

Accuracy: 0.763


# 3. Решение задания с помощью модели RuGPT3 large

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

In [18]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device);

cuda


In [21]:
def calc_loss(text):
    with torch.no_grad():
        inputs = tokenizer.encode(text, return_tensors='pt').reshape(-1).to(device)
        # Создаем батч из одного элемента
        inputs = inputs.unsqueeze(dim=0)
        loss = model(input_ids=inputs, labels=inputs).loss.item()
        return loss
calc_loss('Предложение корректное?')

5.57512092590332

## 3.1.Zero shot

In [26]:
#zero shot
from tqdm import tqdm
tqdm.pandas()

def shot(start: str, text: str, end: list):
    first = ' '.join([start, text, end[0]])
    second = ' '.join([start, text, end[1]])

    loss_1, loss_2 =  calc_loss(first), calc_loss(second)
    return 1 if loss_1 > loss_2 else 0

y_pred = test['sentence'].progress_apply(lambda x: shot('Проверь корректность предложения', x, ['Это предложение корректное', 'Это предложение некорректное']))

100%|██████████| 983/983 [00:40<00:00, 24.30it/s]


In [27]:
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.718819



In [32]:
y_pred = test['sentence'].progress_apply(lambda x: shot('Если ли здесь ошибка?', x, ['Правильно', 'Ошибка']))

100%|██████████| 983/983 [00:45<00:00, 21.78it/s]


In [33]:
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n') # Есть разница с предыдущей затравкой

F1-score = 0.853473



In [30]:
y_pred = test['sentence'].progress_apply(lambda x: shot('Если ли здесь ошибка?', x, ['Верно', 'Неверно']))

100%|██████████| 983/983 [00:43<00:00, 22.35it/s]


In [31]:
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.854312



Выводы: максимальную точность дала затравка: "Верно / Неверно"

## 3.2. Few shot

In [34]:
# 1 shot
promt = """Проверь корректность предложения:
Вдруг решетка беззвучно поехала в сторону, и на балконе возникла таинственная фигура, прячущаяся от лунного света, и погрозила Ивану пальцем. => Верно
"""
y_pred = test['sentence'].apply(lambda x: shot(promt, x, ['=> Верно', '=> Неверно']))
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.855263



In [35]:
# 2 shots
promt = """Проверь корректность предложения:
Вдруг решетка беззвучно поехала в сторону, и на балконе возникла таинственная фигура, прячущаяся от лунного света, и погрозила Ивану пальцем. => Верно
Этим летом не никуда ездили. => Неверно
"""
y_pred = test['sentence'].apply(lambda x: shot(promt, x, ['=> Верно', '=> Неверно']))
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.854312



In [36]:
# 3 shots
promt = """Проверь корректность предложения:
Вдруг решетка беззвучно поехала в сторону, и на балконе возникла таинственная фигура, прячущаяся от лунного света, и погрозила Ивану пальцем. => Верно
Этим летом не никуда ездили. => Неверно
Только Иван выразил какую бы то ни было готовность помочь. => Верно
"""
y_pred = test['sentence'].apply(lambda x: shot(promt, x, ['=> Верно', '=> Неверно']))
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.854312



In [37]:
# 4 shots
promt = """Проверь корректность предложения:
Вдруг решетка беззвучно поехала в сторону, и на балконе возникла таинственная фигура, прячущаяся от лунного света, и погрозила Ивану пальцем. => Верно
Этим летом не никуда ездили. => Неверно
Только Иван выразил какую бы то ни было готовность помочь. => Верно
Теперь ты видишь собственными глазами, как тут хорошо. => Верно
"""
y_pred = test['sentence'].apply(lambda x: shot(promt, x, ['=> Верно', '=> Неверно']))
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

F1-score = 0.854312



### Выводы: точность F1-score не зависит от количества shorts. Отличается только для shot = 1

# 4. Обучение и тестирование модели ruT5 на данной задаче

In [1]:
# !pip install --upgrade pip

In [2]:
# !pip uninstall transformers
# !pip install --no-cache-dir transformers sentencepiece

In [38]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base", use_fast=False)
model = T5ForConditionalGeneration.from_pretrained("ai-forever/ruT5-base")

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [39]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device);

In [40]:
class EvalDataset(Dataset):

    def __init__(self, X):
        self.text = X.reset_index(drop=True)

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=45)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1).to(device) for k, v in output.items()}

class TrainDataset(Dataset):

    def __init__(self, X, label):
        self.text = X.reset_index(drop=True)
        self.label = label.reset_index(drop=True)

    def tokenize(self, text, length=45):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=length)

    def __len__(self):
        return self.label.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        output = {k: v.reshape(-1).to(device) for k, v in output.items()}

        label = 'верно' if self.label[index] == 1 else 'неверно'
        label = self.tokenize(label, length=2).input_ids.reshape(-1).to(device)

        output.update({'labels': label})
        return output

train_ds = TrainDataset(train['sentence'], train['acceptable'])
train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True)

eval_ds = TrainDataset(val['sentence'], val['acceptable'])
eval_dataloader = DataLoader(eval_ds, batch_size=32)

test_ds = EvalDataset(test['sentence'])
test_dataloader = DataLoader(test_ds, batch_size=32)

In [41]:
optimizer = Adam(model.parameters(), lr=1e-5)

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [46]:
def train_model(train_dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} \n ***********************')
        for n_batch, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            if n_batch % 50 == 0:
                loss_train, current = loss.item(), n_batch * batch['input_ids'].shape[0]
                print(f"Loss train: {loss_train:>7f}  [{current:>5d}/{len(train_ds):>5d}]")
                print('Evaluating...')
                loss_val, _ = test_model(eval_dataloader, eval=True)
                print(f"Loss test: {loss_val:>7f}\n")
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

def test_model(test_dataloader, eval=False):
    model.eval()
    y_pred = np.array([])
    y_true = np.array([])
    loss = []
    for n_batch, batch in enumerate(test_dataloader):
        if not eval:
            gen_tok = model.generate(**batch)
            gen_tok = [1 if 2937 in i else 0 for i in gen_tok]  # tokenizer.decode(2937) == 'верно'
            y_true = np.hstack([y_true, gen_tok])
        else:
            outputs = model(**batch)
            loss.append(outputs.loss.item())
    if not eval:
        return y_true
    else:
        return np.sum(loss)/len(loss), y_true

In [47]:
train_model(train_dataloader, num_epochs)

Epoch 1 
 ***********************
Loss train: 2.491924  [    0/ 6295]
Evaluating...
Loss test: 0.298630

Loss train: 0.232251  [ 1600/ 6295]
Evaluating...
Loss test: 0.298630

Loss train: 0.257327  [ 3200/ 6295]
Evaluating...
Loss test: 0.298630

Loss train: 0.257101  [ 4800/ 6295]
Evaluating...
Loss test: 0.298630

Epoch 2 
 ***********************
Loss train: 0.293871  [    0/ 6295]
Evaluating...
Loss test: 0.298630

Loss train: 0.204200  [ 1600/ 6295]
Evaluating...
Loss test: 0.298630

Loss train: 0.288268  [ 3200/ 6295]
Evaluating...
Loss test: 0.298630

Loss train: 0.247213  [ 4800/ 6295]
Evaluating...
Loss test: 0.298630



In [48]:
y_pred = test_model(test_dataloader, eval=False)
print(f'F1-score = {f1_score(y_pred, test["acceptable"]):>3f}\n')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


F1-score = 0.853238



# 5. Сравнение результатов

<table>
    <thead>
        <tr>
            <td>Model</td>
            <td>F1 score</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>ruBERT-base</td>
            <td>0.862374</td>
        </tr>
        <tr>
            <td>ruGPT3-large best zero shot</td>
            <td>0.854312</td>
        </tr>
        <tr>
            <td>ruGPT3-large best few shot</td>
            <td>0.854312</td>
        </tr>
        <tr>
            <td>ruT5-base</td>
            <td>0.853238</td>
        </tr>
    </tbody>
</table>


Были рассмотрены следующие трансформенные модели:
- ruBert-base,
- ruGPT3-large (few-/zero-shot)
- ruT5-base

Как ни странно, но лучший результат был получен на модели ruBert-base (F1 = 0,86; Accuracy = 0.76). В целом, модели показали близкие по точности результаты.

При этом самый простой вариант файтюнинга с помощью few-shot для модели ruGPT3.